In [7]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

Load dataset of choice and get train/test split

In [8]:
import ipynb.fs.full.TrainTest as TrainTest

p = 30 # want this to be high
w = 4  # want this to be low
cw = 9 # want this to be high
# data = np.load('./data/' + 'p' + str(p) + '_w' + str(w) + '_cw' + str(cw) + '.npy')
data = np.load('./data/created/nperseg=sqrt/p29_a[2]_w4_cw9.npy')

x_train, y_train, x_test, y_test = TrainTest.get_train_test(data, 0.8)

Do RandomizedSearchCV

In [9]:
rf = RandomForestClassifier()

param_options = {
    'n_estimators' : range(100, 2001, 100),
    # 'criterion' : [],
    'max_features' : ['auto', 3, 4, 5],
    'max_depth' : [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4],
    # 'min_weight_fraction_leaf' : [],
    # 'max_leaf_nodes' : [],
    # 'min_impurity_split' : [],
    # 'min_impurity_decrease' : [],
    'bootstrap' : [True, False]
    # 'oob_score' : [],
}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_options, 
    n_iter = 100, cv =10, verbose=1, random_state=42, n_jobs = -1)

rf_random.fit(x_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 24.2min finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'min_samples_leaf': [1, 2, 4], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 3, 4, 5], 'n_estimators': range(100, 2001, 100)},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [10]:
best = rf_random.best_params_
best

{'bootstrap': False,
 'max_depth': 40,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600}

In [11]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

rf = RandomForestClassifier(**best)

avg_acc = 0
avg_prec = 0
avg_f1 = 0
num = 20
for i in range(num):
    rf.fit(x_train, y_train)
    
    y_pred = rf.predict(x_test)
    
    avg_acc += accuracy_score(y_test, y_pred)
    avg_prec += precision_score(y_test, y_pred)
    avg_f1 += f1_score(y_test, y_pred)

avg_acc /= num
avg_prec /= num
avg_f1 /= num

print('Average accuracy:  ' + str(avg_acc))
print('Average precision: ' + str(avg_prec))
print('Average F1 Score:  ' + str(avg_f1))

Average accuracy:  0.7662025316455698
Average precision: 0.8102802210423927
Average F1 Score:  0.774768758337344
